# Comores DroneMD (Metadata) Notebook

### Common variables:

In [23]:
# Script folder
dronemdt_folder = '/home/sylvain/DroneMDT'
# Session / Flight data input
input = '/home/sylvain/DATA/COMORES'
# Output geoflow config (json and csv entities)
output = '/home/sylvain/DATA/COMORES'
# Geoflow folder where .env is stored and jobs will be available
geoflow_folder = '/home/sylvain/geoflow-g2oi'
# Geoserver configuration
WS = 'DGSC'
DS = 'comores'
# Zenodo setup
sandbox = False
# Move to dronemdt_folder
import os
os.chdir(dronemdt_folder)

## Part 1a: Check images validity

In [2]:
# DATA have to be copied from Drone internal storage or sdcard DCIM folder using suh naming convention:
# YYYYMMDD_COUNTRYCODE-optionalplace_device_session-number
# ├── DCIM :  folder to store videos and photos depending on the media collected
#
# Check validity of images copied from Drone to disk

In [3]:
from check_images import *
img_list = im_list(input)
for i in range(len(img_list)):
    check_image_with_pil(img_list[i])

## Part 1b: Generate report

In [4]:
## Using subprocess:
import subprocess
subprocess.check_output(["Rscript", "./generate_drone_metadata.R", input])
# from rpy2.robjects.packages import importr


Loading required package: magrittr
uasimg (version 1.8.3)
Bug reports: https://github.com/ucanr-igis/uasimg/issues
/home/sylvain/DATA/COMORES/20230626_COM-bandamaji_UAV-01_1/DCIM
 - Looking for image files (all image files)
 - Found DJI Mavic 2 Pro Hasselblad
 - Running exiftool (this can take a while)...Done.
 - Total file size: 926 MB
 - Creating footprints...Done.
 - Computing forward overlap...Done.
 - Cache saved
 - Metadata file not found
All done
2023-06-26_15-34-52_102JPEGs
 - computing thumbnail file names...Done.
 - thumbnails already exist
 - Done.


processing file: uas_report_metrics.Rmd
output file: uas_report_metrics.knit.md


Output created: METADATA/Report_2023-06-26_15-34-52_102JPEGs.html
Done.
/home/sylvain/DATA/COMORES/20230626_COM-bandamaji_UAV-01_2/DCIM
 - Looking for image files (all image files)
 - Found DJI Mavic 2 Pro Hasselblad
 - Running exiftool (this can take a while)...Done.
 - Total file size: 884 MB
 - Creating footprints...Done.
 - Computing forward ov

b'[1] "/home/sylvain/DATA/COMORES"\n[1] "processing: /home/sylvain/DATA/COMORES/20230626_COM-bandamaji_UAV-01_1"\n1/27                   \n2/27 [unnamed-chunk-1] \n3/27                   \n4/27 [unnamed-chunk-2] \n5/27                   \n6/27 [setup]           \n7/27                   \n8/27 [data_uris_insert]\n9/27                   \n10/27 [show_flight_info]\n11/27                   \n12/27 [leaflet_ctr]     \n13/27                   \n14/27 [fp_tab]          \n15/27                   \n16/27 [leaflet_fp]      \n17/27                   \n18/27 [kml_links]       \n19/27                   \n20/27 [nothingtodo]     \n21/27                   \n22/27 [unnamed-chunk-3] \n23/27                   \n24/27 [unnamed-chunk-4] \n25/27                   \n26/27 [unnamed-chunk-5] \n27/27                   \n/usr/bin/pandoc +RTS -K512m -RTS uas_report_metrics.knit.md --to html4 --from markdown+autolink_bare_uris+tex_math_single_backslash --output /home/sylvain/DATA/COMORES/20230626_COM-bandamaji_UA

## Part 2: Generate metadata

In [74]:
# Import packages and modules
import os
import shutil
import pandas as pd
import geopandas as gpd
import datetime
# from IPython.display import display
import argparse
import json
import pycountry
os.chdir(dronemdt_folder)
from meteo_helper import meteo
from read_metadata import *
from exif_helper import *
from enriched_func import *
from zenodo_helper import *
from zip_helper import *
from geoflow_helper import *
from raster_helper import *
from process_webodm import *
from settings import *

In [4]:
### Filter foder name
folder_name = filter_folder_name(input, 'UAV')
folder_name

['20230626_COM-bangoua-kouni_UAV-02_1',
 '20230701_COM-nioumadzaha-mitsoudje_UAV-02_1',
 '20230626_COM-bandamaji_UAV-01_2',
 '20230701_COM-nioumadzaha-mitsoudje_UAV-01_3',
 '20230626_COM-mitsamiouli_UAV-01_1',
 '20230701_COM-nioumadzaha-mitsoudje_UAV-01_1',
 '20230627_COM-vouvouni_UAV-02_2',
 '20230701_COM-nioumadzaha-mitsoudje_UAV-01_2',
 '20230627_COM-vouvouni_UAV-01_1',
 '20230627_COM-vouvouni_UAV-02_1',
 '20230626_COM-mitsamiouli_UAV-02_1',
 '20230627_COM-vouvouni_UAV-01_2',
 '20230626_COM-bangoua-kouni_UAV-01_1',
 '20230626_COM-bandamaji_UAV-02_1',
 '20230626_COM-bandamaji_UAV-01_1',
 '20230628_COM-maloudja_UAV-02_2',
 '20230626_COM-mitsamiouli_UAV-01_2']

In [5]:
### Put data in basic panda dataframe
df = {}
df = basic_geoflow_df(folder_name)
df.head()

,Identifier,Title,Description,Subject,Creator,Date,Type,Language,SpatialCoverage,TemporalCoverage,Relation,Rights,Provenance,Format,Data
0,20230626_COM-bangoua-kouni_UAV-02_1,title:20230626 com bangoua kouni uav 02 1,abstract:'No \n\ndescription\n!',project:None,creator:None,publication:2023-10-11_\nedition:2023-10-11,dataset,eng,,COM-bangoua-kouni,None,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,None,resource:image/jpg,None
1,20230701_COM-nioumadzaha-mitsoudje_UAV-02_1,title:20230701 com nioumadzaha mitsoudje uav 02 1,abstract:'No \n\ndescription\n!',project:None,creator:None,publication:2023-10-11_\nedition:2023-10-11,dataset,eng,,COM-bangoua-kouni,None,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,None,resource:image/jpg,None
2,20230626_COM-bandamaji_UAV-01_2,title:20230626 com bandamaji uav 01 2,abstract:'No \n\ndescription\n!',project:None,creator:None,publication:2023-10-11_\nedition:2023-10-11,dataset,eng,,COM-bangoua-kouni,None,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,None,resource:image/jpg,None
3,20230701_COM-nioumadzaha-mitsoudje_UAV-01_3,title:20230701 com nioumadzaha mitsoudje uav 01 3,abstract:'No \n\ndescription\n!',project:None,creator:None,publication:2023-10-11_\nedition:2023-10-11,dataset,eng,,COM-bangoua-kouni,None,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,None,resource:image/jpg,None
4,20230626_COM-mitsamiouli_UAV-01_1,title:20230626 com mitsamiouli uav 01 1,abstract:'No \n\ndescription\n!',project:None,creator:None,publication:2023-10-11_\nedition:2023-10-11,dataset,eng,,COM-bangoua-kouni,None,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,None,resource:image/jpg,None


In [6]:
### Define Metadata file
md_file = 'metadata.txt'
### Parse global metadata
mdt_df_glob = global_mdt(input, md_file)
#### Enrich metadata
enrich_drone_mdt(input, df, md_file, mdt_df_glob)

INFO: Global `metadata.txt` found !
WARN: No metadata.txt in /home/sylvain/DATA/COMORES/20230626_COM-bangoua-kouni_UAV-02_1/METADATA to parse
INFO: GPS Folder found
Format id to obtain date, country name, optionnal place, and platform
####################
### 2. Enrich Title
####################
INFO: Using glocal title
Images drone {platform}, {place}, {country} - {date}
format field
####################
### 8. Enrich Language
####################
INFO: Using glocal language
####################
### 4. Enrich Subject
####################
INFO: Using glocal theme
INFO: Using glocal project
####################
### 5. Enrich Creator
####################
WARN: No global author
WARN: No author to enrich
INFO: Using glocal publisher
INFO: Using glocal owner
INFO: Using glocal pointOfContact
INFO: Using glocal processor
####################
### 11. Enrich Relation
####################
INFO: Using glocal thumbnail
INFO: Using glocal http
####################
### Reading DCIM folder
#########

/home/sylvain/DroneMDT/enriched_func.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  enriched = df_global.iloc[0][keyword][0]


/home/sylvain/DATA/COMORES/20230626_COM-bangoua-kouni_UAV-02_1
Generate image with a set of images
####################
### 10. Enrich TemporalCoverage
####################
####################
### 00. Prepare Provenance (13) and Description (3) 
####################
/home/sylvain/DATA/COMORES/20230626_COM-bangoua-kouni_UAV-02_1/DCIM/20230627_104301-DJI_0665.jpg
####################
Fetching open-meteo
####################
Location: -11.372115149999999, 43.364221900000004
Begin: 2023-06-27 10-32-22 - End: 2023-06-27 10-49-42
Model: best_match


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2021    0  2021    0     0   6229      0 --:--:-- --:--:-- --:--:--  6218


Convert Dataframe to UTM
Index(['FileName', 'GPSLatitude', 'GPSLongitude', 'GPSAltitude',
       'FlightRollDegree', 'FlightPitchDegree', 'FlightYawDegree',
       'GimbalRollDegree', 'GimbalPitchDegree', 'GimbalYawDegree',
       'ImageWidth', 'ImageLength', 'BitsPerSample', 'ImageDescription',
       'Make', 'Model', 'Orientation', 'SamplesPerPixel', 'XResolution',
       'YResolution', 'ResolutionUnit', 'Software', 'DateTime',
       'YCbCrPositioning', 'ExifOffset', 'GPSVersionID', 'ThumbnailImageWidth',
       'ThumbnailImageLength', 'ThumbnailCompression', 'ThumbnailXResolution',
       'ThumbnailYResolution', 'ThumbnailResolutionUnit',
       'ThumbnailJPEGInterchangeFormat',
       'ThumbnailJPEGInterchangeFormatLength', 'ExposureTime', 'FNumber',
       'ExposureProgram', 'ISOSpeedRatings', 'ExifVersion', 'DateTimeOriginal',
       'DateTimeDigitized', 'ComponentsConfiguration', 'ExposureBiasValue',
       'MaxApertureValue', 'MeteringMode', 'LightSource', 'Flash',
       'Foc

...

Convert Dataframe to UTM
Index(['FileName', 'GPSLatitude', 'GPSLongitude', 'GPSAltitude',
       'FlightRollDegree', 'FlightPitchDegree', 'FlightYawDegree',
       'GimbalRollDegree', 'GimbalPitchDegree', 'GimbalYawDegree',
       'ImageWidth', 'ImageLength', 'BitsPerSample', 'ImageDescription',
       'Make', 'Model', 'Orientation', 'SamplesPerPixel', 'XResolution',
       'YResolution', 'ResolutionUnit', 'Software', 'DateTime',
       'YCbCrPositioning', 'ExifOffset', 'GPSVersionID', 'ThumbnailImageWidth',
       'ThumbnailImageLength', 'ThumbnailCompression', 'ThumbnailXResolution',
       'ThumbnailYResolution', 'ThumbnailResolutionUnit',
       'ThumbnailJPEGInterchangeFormat',
       'ThumbnailJPEGInterchangeFormatLength', 'ExposureTime', 'FNumber',
       'ExposureProgram', 'ISOSpeedRatings', 'ExifVersion', 'DateTimeOriginal',
       'DateTimeDigitized', 'ComponentsConfiguration', 'ExposureBiasValue',
       'MaxApertureValue', 'MeteringMode', 'LightSource', 'Flash',
       'Foc

/home/sylvain/DroneMDT/enriched_func.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  enriched = df_global.iloc[0][keyword][0]


/home/sylvain/DATA/COMORES/20230626_COM-mitsamiouli_UAV-01_2
Generate image with a set of images
####################
### 10. Enrich TemporalCoverage
####################
####################
### 00. Prepare Provenance (13) and Description (3) 
####################
/home/sylvain/DATA/COMORES/20230626_COM-mitsamiouli_UAV-01_2/DCIM/20230627_123440_DJI_0196.JPG
####################
Fetching open-meteo
####################
Location: -11.390653799999999, 43.2888092
Begin: 2023-06-27 12-30-25 - End: 2023-06-27 12-44-19
Model: best_match


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2022    0  2022    0     0   7505      0 --:--:-- --:--:-- --:--:--  7516


Convert Dataframe to UTM
Index(['FileName', 'GPSLatitude', 'GPSLongitude', 'GPSAltitude',
       'FlightRollDegree', 'FlightPitchDegree', 'FlightYawDegree',
       'GimbalRollDegree', 'GimbalPitchDegree', 'GimbalYawDegree',
       'ImageWidth', 'ImageLength', 'BitsPerSample', 'ImageDescription',
       'Make', 'Model', 'Orientation', 'SamplesPerPixel', 'XResolution',
       'YResolution', 'ResolutionUnit', 'Software', 'DateTime',
       'YCbCrPositioning', 'ExifOffset', 'GPSVersionID', 'ThumbnailImageWidth',
       'ThumbnailImageLength', 'ThumbnailCompression', 'ThumbnailXResolution',
       'ThumbnailYResolution', 'ThumbnailResolutionUnit',
       'ThumbnailJPEGInterchangeFormat',
       'ThumbnailJPEGInterchangeFormatLength', 'ExposureTime', 'FNumber',
       'ExposureProgram', 'ISOSpeedRatings', 'ExifVersion', 'DateTimeOriginal',
       'DateTimeDigitized', 'ComponentsConfiguration', 'ExposureBiasValue',
       'MaxApertureValue', 'MeteringMode', 'LightSource', 'Flash',
       'Foc

In [7]:
# Set prefix for generated json and csv entities
out_prefix = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
out_prefix

'20231011-102200'

In [8]:
# Visualize output
df.head()

,Identifier,Title,Description,Subject,Creator,Date,Type,Language,SpatialCoverage,TemporalCoverage,Relation,Rights,Provenance,Format,Data
0,20230626_COM-bangoua-kouni_UAV-02_1,"title:Images drone UAV, Bangoua kouni, Comoros - 20230626",abstract:Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Bangoua...,"theme[General]:Comores,Grande Comore,flooding,natural risk,risque inondation,drone,uav,uas,DEM,DSM,or...",author:_\npublisher:sylvain.poulain@ird.fr_\nowner:comores.dgsc@gmail.com_\npointOfContact:julien.bar...,publication:2023-10-11_\nedition:2023-10-11,dataset,fra,,2023-06-27 10-32-22 - 2023-06-27 10-49-42,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/jpg_\ndistribution:application/geopackage+sqlite3[GeoPackage],source:SurveyMetadata.gpkg@/home/sylvain/DATA/COMORES/20230626_COM-bangoua-kouni_UAV-02_1/GPS/SurveyM...
1,20230701_COM-nioumadzaha-mitsoudje_UAV-02_1,"title:Images drone UAV, Nioumadzaha mitsoudje, Comoros - 20230701",abstract:Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Nioumad...,"theme[General]:Comores,Grande Comore,flooding,natural risk,risque inondation,drone,uav,uas,DEM,DSM,or...",author:_\npublisher:sylvain.poulain@ird.fr_\nowner:comores.dgsc@gmail.com_\npointOfContact:julien.bar...,publication:2023-10-11_\nedition:2023-10-11,dataset,fra,,2023-06-26 15-59-22 - 2023-07-01 11-36-44,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/jpg_\ndistribution:application/geopackage+sqlite3[GeoPackage],source:SurveyMetadata.gpkg@/home/sylvain/DATA/COMORES/20230701_COM-nioumadzaha-mitsoudje_UAV-02_1/GPS...
2,20230626_COM-bandamaji_UAV-01_2,"title:Images drone UAV, Bandamaji, Comoros - 20230626",abstract:Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Bandama...,"theme[General]:Comores,Grande Comore,flooding,natural risk,risque inondation,drone,uav,uas,DEM,DSM,or...",author:_\npublisher:sylvain.poulain@ird.fr_\nowner:comores.dgsc@gmail.com_\npointOfContact:julien.bar...,publication:2023-10-11_\nedition:2023-10-11,dataset,fra,,2023-06-26 15-51-39 - 2023-06-26 15-58-55,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/jpg_\ndistribution:application/geopackage+sqlite3[GeoPackage],source:SurveyMetadata.gpkg@/home/sylvain/DATA/COMORES/20230626_COM-bandamaji_UAV-01_2/GPS/SurveyMetad...
3,20230701_COM-nioumadzaha-mitsoudje_UAV-01_3,"title:Images drone UAV, Nioumadzaha mitsoudje, Comoros - 20230701",abstract:Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Nioumad...,"theme[General]:Comores,Grande Comore,flooding,natural risk,risque inondation,drone,uav,uas,DEM,DSM,or...",author:_\npublisher:sylvain.poulain@ird.fr_\nowner:comores.dgsc@gmail.com_\npointOfContact:julien.bar...,publication:2023-10-11_\nedition:2023-10-11,dataset,fra,,2023-07-01 11-26-01 - 2023-07-01 11-34-29,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L

In [9]:
# Visualize Description
print(df.iloc[0]['Description'])

abstract:Jeu de données composé d’images de drone DJI Mavic 2 Pro UAV acquises sur le site de Bangoua kouni, Grande Comore, Comoros à la date suivante : 20230626.
<br />Les vols ont été réalisés en partenariat avec la DGSC (Direction Générale de la Sécurité Civile, Comores) dans le but de créer des modèles numériques d'élévations pour cartographier les risques naturels d’inondations ou de submersion marine.
<br />
<br /><b>Le dépôt est composé des éléments suivants:</b>
<br />    - 00_: Planche d'aperçu des images
<br />    - DCIM.zip: Images brutes issues du drone
<br />    - GPS.zip: Geopackage contenant l’emprise du survol ainsi que la géolocalisation des images accompagnées de leurs miniatures dans la table d’attribut en base64
<br />    - METADATA.zip: Métadonnées au format ISO19115, Rapports avec miniatures des images de drone (dossier tb) et statistiques de vols.
<br />
<br /><b>Arborescence d'origine:</b>
<br />└─ 20230626_COM-bangoua-kouni_UAV-02_1
<br />  └─ DCIM
<br />  └─ G

In [10]:
##############
#### ISO 19115 config
##############
## Export to csv
df.to_csv(os.path.join(output, out_prefix + "_iso19115-metadata.csv"), quoting=1, quotechar='"', index=False)
## Generate json
iso19115config = {'profile': {'id': 'iso19115', 
                              'name': 'iso19115', 
                              'project': 'Generate iso19115 with Geoflow', 
                              'organization': 'IRD', 
                              'environment': {'file': '.env', 
                                              'hide_env_vars': ['MOTDEPASSE']}, 
                              'logos': ['https://en.ird.fr/sites/ird_fr/files/2019-08/logo_IRD_2016_BLOC_UK_COUL.png'], 
                              'mode': 'entity'}, 
                  'metadata': {'entities': 
                               [{'handler': 'csv', 
                                 'source': os.path.join(output, out_prefix) + "_iso19115-metadata.csv"}], 
                               'contacts': 
                               [{'handler': 'csv', 
                                 'source': 'https://drive.ird.fr/s/EYS3qccyB28PrA9/download/geoflow_g2oi_contacts.csv'}]}, 
                  'software': [], 
                  'actions': 
                  [{'id': 'geometa-create-iso-19115',
                    'options' : 
                    {"logo": True}, 
                    "run": True}]}

# Export json to file
with open(os.path.join(output, out_prefix + '_iso19115-config.json'), 'w') as f:
    json.dump(iso19115config, f, indent=4)

In [9]:
# Optional : could start from here => reload dataframe from csv
# df = {}
# df = pd.read_csv(output, out_prefix + "_iso19115-metadata.csv")

In [12]:
### Execute geoflow
xml19115 = run_geoflow(os.path.join(output, out_prefix + '_iso19115-config.json'), geoflow_folder)
os.chdir(dronemdt_folder)

R[write to console]: Rows: 129 Columns: 14

R[write to console]: ── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (14): Identifier, Email, OrganizationName, PositionName, LastName, First...

R[write to console]: 
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

R[write to console]: Rows: 17 Columns: 15

R[write to console]: ── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (15): Identifier, Title, Description, Subject, Creator, Date, Type, Lang...

R[write to console]: 
ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

R[write to console]: Loading required package: geometa

R[write to console]: Loading required package: ows4R



In [13]:
# Geoflow job log folder
print(xml19115[0])

/home/sylvain/geoflow-g2oi/jobs/20231011110021


In [14]:
## Copy generated ISO19115 to original metadata folder
for id in range(len(df)):
    jobs19115 = xml19115[0] + '/entities/' + df.iloc[id]['Identifier'] + '/metadata/'
    fname19115 = df.iloc[id]['Identifier'] + '_ISO-19115.xml'
    if fname19115 in os.listdir(jobs19115):
        shutil.copyfile(jobs19115+fname19115, 
                        input + "/" + df.iloc[id]['Identifier'] + '/METADATA/' + fname19115)

## Part 3. Upload RAW data to Zenodo

### ZIP all folders

In [15]:
### Zip all folders
zip_each_folder(input, df)
### Reference Zip's in Dataframe
for zip in range(len(df)):
    source = []
    sess_path = os.path.join(input, df.iloc[zip]['Identifier'])
    source = os.listdir(sess_path)
    if len(source) < 1:
        pass
    else:
        source_file = []
        for file in range(len(source)):
            if "PROCESSED_DATA.zip" in source:
                pass
            else:
                if os.path.isfile(os.path.join(sess_path, source[file])):
                    if len(source) < 2:
                        df.iloc[zip, df.columns.get_loc('Data')] = "source:" + source[file] + "@" + sess_path + "/" + source[file] + "_\n" + "sourceType:other_\nsourceZip:false"
                        source_file = source[file]
                    else:
                        source_file.append(source[file])
                        # df.iloc[zip, df.columns.get_loc('Data')] = "source:" + source[file] + "@" + sess_path + source[file] + "_\n" + "sourceType:other_\nsourceZip:false"
        if ".zip" in  df.iloc[zip, df.columns.get_loc('Data')]:
            pass
        else:
            df.iloc[zip, df.columns.get_loc('Data')] = "source:"
            for i in range(len(source_file)):
                df.iloc[zip, df.columns.get_loc('Data')] =  str(df.iloc[zip, df.columns.get_loc('Data')]) + source_file[i] + "@" + sess_path + "/" + source_file[i] + ","
    
    if "," in str(df.iloc[zip, df.columns.get_loc('Data')])[-1]:
        df.iloc[zip, df.columns.get_loc('Data')] = str(df.iloc[zip, df.columns.get_loc('Data')])[:-1] + "_\n" + "sourceType:other_\nupload:true"


INFO: Ziping 20230626_COM-bangoua-kouni_UAV-02_1
    INFO: 00_sample_rawdata_overview.png is not a folder
INFO: Ziping 20230701_COM-nioumadzaha-mitsoudje_UAV-02_1
    INFO: 00_sample_rawdata_overview.png is not a folder
INFO: Ziping 20230626_COM-bandamaji_UAV-01_2
    INFO: 00_sample_rawdata_overview.png is not a folder
INFO: Ziping 20230701_COM-nioumadzaha-mitsoudje_UAV-01_3
    INFO: 00_sample_rawdata_overview.png is not a folder
INFO: Ziping 20230626_COM-mitsamiouli_UAV-01_1
    INFO: 00_sample_rawdata_overview.png is not a folder
INFO: Ziping 20230701_COM-nioumadzaha-mitsoudje_UAV-01_1
    INFO: 00_sample_rawdata_overview.png is not a folder
INFO: Ziping 20230627_COM-vouvouni_UAV-02_2
    INFO: 00_sample_rawdata_overview.png is not a folder
INFO: Ziping 20230701_COM-nioumadzaha-mitsoudje_UAV-01_2
    INFO: 00_sample_rawdata_overview.png is not a folder
INFO: Ziping 20230627_COM-vouvouni_UAV-01_1
    INFO: 00_sample_rawdata_overview.png is not a folder
INFO: Ziping 20230627_COM-vouv

### Generate config files

In [16]:
### Export raw data df to csv
fname = out_prefix + '_zenodo-rawdata'
df.to_csv(os.path.join(output, fname + ".csv"), quoting=1, quotechar='"', index=False)

### Configure and export Zenodo json
# SANDBOX
# zen_json = zenodo_json(fname, 'sandbox', output, out_prefix)
# PRODUCTION
zen_json = zenodo_json(fname, 'no_sandbox', output, out_prefix)
with open(os.path.join(output, out_prefix + '_rawfiles-config.json'), 'w') as f:
    json.dump(zen_json, f, indent=4)


### Execute Geoflow to upload to Zenodo

In [23]:
### Execute geoflow :: upload to geoflow
# upload_raw_to_zenodo = 
config_raw = os.path.join(output, out_prefix + '_rawfiles-config.json')
# run_geoflow(config_raw, geoflow_folder)
os.chdir(geoflow_folder)
# subprocess.run(["R", "-e", "geoflow::initWorkflow('{}')".format(config_raw)], capture_output=False)
subprocess.call(["R", "-e", "geoflow::executeWorkflow('{}')".format(config_raw)],
                stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT,)
os.chdir(dronemdt_folder)

## Part 4. Download from ODM

In [34]:
# Take csv generated from geoflow's zenodo upload as updated dataframe
df = {}
df = pd.read_csv(os.path.join(input, 'zenodo_entities_1_with_pids_for_publication.csv'))
df = pd.read_csv(os.path.join(input, 'comores_processed.csv'))
df.head()

,Identifier,Title,Description,Subject,Creator,Date,Type,Language,SpatialCoverage,TemporalCoverage,Relation,Rights,Provenance,Format,Data
0,id:20230626_COM-bangoua-kouni_UAV-02_1_\ndoi:10.5281/zenodo.8431136_\nconceptdoi:10.5281/zenodo.8431135,"title:""Orthophoto & DEM (MNE) issues d'images de drone, Bangoua kouni, Comores - 20230626""","abstract:""Ce jeu de données présente les résultats des traitements photogrammétriques d’images de dro...","theme[General]:Comores,Grande Comore,flooding,natural risk,risque inondation,drone,uav,uas,DEM,DSM,or...","author:sylvain.poulain@ird.fr,julien.barde@ird.fr,natali.mmadi@gmail.com,mohamedhamidou957@gmail.com,...",publication:2023-10-11_\nedition:2023-10-11,dataset,fra,NaN,2023-06-27,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/jpg_\ndistribution:application/geopackage+sqlite3[GeoPackage],"source:METADATA.zip@/home/sylvain/DATA/COMORES/20230626_COM-bangoua-kouni_UAV-02_1/METADATA.zip,GPS.z..."
1,id:20230701_COM-nioumadzaha-mitsoudje_UAV-02_1_\ndoi:10.5281/zenodo.8431203_\nconceptdoi:10.5281/zeno...,"title:""Orthophoto & DEM (MNE) issues d'images de drone, Nioumadzaha mitsoudje, Comores - 20230701""","abstract:""Ce jeu de données présente les résultats des traitements photogrammétriques d’images de dro...","theme[General]:Comores,Grande Comore,flooding,natural risk,risque inondation,drone,uav,uas,DEM,DSM,or...","author:sylvain.poulain@ird.fr,julien.barde@ird.fr,natali.mmadi@gmail.com,mohamedhamidou957@gmail.com,...",publication:2023-10-11_\nedition:2023-10-11,dataset,fra,NaN,2023-06-26 15-59-22 - 2023-07-01 11-36-44,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/jpg_\ndistribution:application/geopackage+sqlite3[GeoPackage],source:METADATA.zip@/home/sylvain/DATA/COMORES/20230701_COM-nioumadzaha-mitsoudje_UAV-02_1/METADATA.z...
2,id:20230626_COM-bandamaji_UAV-01_2_\ndoi:10.5281/zenodo.8431498_\nconceptdoi:10.5281/zenodo.8431497,"title:""Orthophoto & DEM (MNE) issues d'images de drone, Bandamaji, Comores - 20230626""","abstract:""Ce jeu de données présente les résultats des traitements photogrammétriques d’images de dro...","theme[General]:Comores,Grande Comore,flooding,natural risk,risque inondation,drone,uav,uas,DEM,DSM,or...","author:sylvain.poulain@ird.fr,julien.barde@ird.fr,natali.mmadi@gmail.com,mohamedhamidou957@gmail.com,...",publication:2023-10-11_\nedition:2023-10-11,dataset,fra,NaN,2023-06-26 15-51-39 - 2023-06-26 15-58-55,thumbnail:G2OI@https://g2oi.ird.fr/static/img/logo_ird.png_\nhttp:Website G2OI[G2OI Website]@https://...,useLimitation:Utilisation libre sous réserve de mentionner la source (a minima le nom du producteur) ...,"statement:""- Camera model and parameters:\n Make: Hasselblad\n Model: L1D-20c\n Width: 5472\n Height:...",resource:image/jpg_\ndistribution:application/geopackage+sqlite3[GeoPackage],"source:METADATA.zip@/home/sylvain/DATA/COMORES/20230626_COM-bandamaji_UAV-01_2/METADATA.zip,GPS.zip@/..."
3,id:20230701_COM-nioumadzaha-mitsoudje_UAV-01_3_\ndoi:10.5281/zenodo.8431531_\nconceptdoi:10.5281/zeno...,"title:""Orthophoto & DEM (MNE) issues d'images de drone, Nioumadzaha mitsoudje, Comores - 20230701""","abstract:""Ce jeu de données présente les résultats des traitements photogrammétriques d’images de dro...","theme[General]:Comores,Grande Comore,flooding,natural risk,risque inondation,drone,uav,uas,DEM,DSM,or...","author:sylvain.poulain@ird.fr,julien.barde@ird.fr,natali.mmadi@gmail.com,mohamedhamidou9

In [ ]:
### SEARCH AND DOWNLOAD FROM ODM
odm_projects = list_all_projects()
for projects in range(len(odm_projects)):
    for p in range(len(df)):
        if df.iloc[p]['Identifier'].split('_\n')[0].split(':')[1] in odm_projects[projects]['name']:
            print(projects, odm_projects[projects]['id'], odm_projects[projects]['name'], odm_projects[projects]['tasks'])
            project_id = odm_projects[projects]['id']
            tasks =  odm_projects[projects]['tasks']
            res = requests.post(settings.SERVER + '/api/token-auth/', 
                            data={'username': settings.USERNAME,
                                'password': settings.PASSWORD}).json()
            if 'token' in res:
                print("Still logged-in!")
                token = res['token']
                for task in range(len(tasks)):
                    task_id = tasks[task]
                    print(task_id)
                    res = requests.get(settings.SERVER + "/api/projects/{}/tasks/{}/download/all.zip".format(project_id, task_id), 
                                    headers={'Authorization': 'JWT {}'.format(token)},
                                    stream=True)
                    # token = res['token']
                    sess_path = os.path.join(input, df.iloc[p]['Identifier'].split('_\n')[0].split(':')[1])
                    ### Reading PROCESSED_DATA folder
                    ### Check and create PROCESSED_DATA Directory if it not exists
                    # process_dir=os.path.join(sess_path, 'PROCESSED_DATA')
                    # if os.path.exists(process_dir):
                    if os.path.exists(sess_path):
                        if 'PROCESSED_DATA.zip' in os.listdir(sess_path):
                            print("INFO: data already found in {} !".format(df.iloc[p]['Identifier'].split('_\n')[0].split(':')[1]))
                        else:
                        #     print("INFO: Create PROCESSED_DATA folder in {} folder")
                        #     os.mkdir(process_dir)
                            print("Downloading all.zip")
                            with open(os.path.join(input, df.iloc[p]['Identifier'].split('_\n')[0].split(':')[1], "all.zip"), 'wb') as f:
                                for chunk in res.iter_content(chunk_size=1024): 
                                    if chunk:
                                        f.write(chunk)
                            # file_newname = "ortho_" + df.iloc[p]['Identifier'].split('_\n')[0].split(':')[1] + ".tif"
                            file_newname = 'PROCESSED_DATA.zip'
                            print("Renaming all.zip to PROCESSED_DATA.zip")
                            os.rename(os.path.join(sess_path, "all.zip"), os.path.join(sess_path, file_newname))
                            print("Saved " + file_newname)

                        res = requests.get(settings.SERVER + "/api/projects/{}/tasks/{}/download/report.pdf".format(project_id, task_id), 
                                            headers={'Authorization': 'JWT {}'.format(token)},
                                            stream=True)
                        pdf_newname = '000_photogrammetry_report.pdf'
                        if pdf_newname in os.listdir(sess_path):
                            print("INFO: PDF already found in {} !".format(df.iloc[p]['Identifier'].split('_\n')[0].split(':')[1]))
                        else:
                            print("Downloading report.pdf")
                            with open(os.path.join(input, df.iloc[p]['Identifier'].split('_\n')[0].split(':')[1], "report.pdf"), 'wb') as pdf:
                                for chunk in res.iter_content(chunk_size=1024): 
                                    if chunk:
                                        pdf.write(chunk)
                            
                            os.rename(os.path.join(sess_path, "report.pdf"), os.path.join(sess_path, pdf_newname))
                            print("Saved " + pdf_newname)                    
            ### Enrich Metadata
            dfdlodm = df
            dfdlodm.iloc[p, dfdlodm.columns.get_loc('Provenance')] = dfdlodm.iloc[p]["Provenance"] + "_\nprocess:Download from WebODM"
            dfdlodm.iloc[p, dfdlodm.columns.get_loc('Format')] = "resource:image/tiff_\ndistribution:image/tiff[GeoTIFF]"
            dfdlodm.iloc[p, dfdlodm.columns.get_loc('Data')] = pdf_newname + "@"+ os.path.join(sess_path, pdf_newname) + "," + file_newname + "@"+ os.path.join(sess_path, file_newname) + "_\nsourceType:other_\nsourceZip:False"

In [64]:
for entities in range(len(df)):
    sess_path = os.path.join(input, df.iloc[entities]['Identifier'].split('_\n')[0].split(':')[1])
    # df.iloc[entities, df.columns.get_loc('Provenance')] = df.iloc[p]["Provenance"] + "_\nprocess:Download from WebODM"
    # df.iloc[entities, df.columns.get_loc('Format')] = "resource:image/tiff_\ndistribution:image/tiff[GeoTIFF]"
    df.iloc[entities, df.columns.get_loc('Data')] = "source:" + pdf_newname + "@"+ os.path.join(sess_path, pdf_newname) + "," + file_newname + "@"+ os.path.join(sess_path, file_newname) + "_\nsourceType:other"

In [66]:
df.iloc[1]["Data"]

'source:000_photogrammetry_report.pdf@/home/sylvain/DATA/COMORES/20230701_COM-nioumadzaha-mitsoudje_UAV-02_1/000_photogrammetry_report.pdf,PROCESSED_DATA.zip@/home/sylvain/DATA/COMORES/20230701_COM-nioumadzaha-mitsoudje_UAV-02_1/PROCESSED_DATA.zip_\nsourceType:other'

In [67]:
### Export processed data df to csv
# fname = out_prefix + '_zenodo-rawdata'
# df.to_csv(os.path.join(output, fname + ".csv"), quoting=1, quotechar='"', index=False)
df.to_csv(os.path.join(output, "comores_processed.csv"), quoting=1, quotechar='"', index=False)

### Geoflow to upload processed data

In [73]:
### Execute geoflow :: upload to geoflow
# config_raw = os.path.join(output, out_prefix + '_rawfiles-config.json')
pdata = os.path.join(output, 'comores_processed.json')
# run_geoflow(config_raw, geoflow_folder)
os.chdir(geoflow_folder)
# subprocess.run(["R", "-e", "geoflow::initWorkflow('{}')".format(config_raw)], capture_output=False)
subprocess.call(["R", "-e", "geoflow::executeWorkflow('{}')".format(pdata)],
                stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT,)
os.chdir(dronemdt_folder)

In [58]:
sess_path

'/home/sylvain/DATA/COMORES/20230626_COM-bandamaji_UAV-01_2'

In [118]:
# Reload Raw Dataframe
%load_ext autoreload
%autoreload 2
# out_prefix ='20231011-102200'
df = {}
df = pd.read_csv(os.path.join(input,'comores_processed-Copy1.csv'))
### RAW DATA
#### df = pd.read_csv('~/geoflow-g2oi/jobs/20231012232907/zenodo/zenodo_entities_1_with_pids_for_publication.csv')
# df = pd.read_csv(os.path.join(input,'20231011-102200_zenodo-rawdata_updated_doi.csv'))
df.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,Identifier,Title,Description,Subject,Creator,Date,Type,Language,SpatialCoverage,TemporalCoverage,Relation,Rights,Provenance,Format,Data
0,id:20230626_COM-bangoua-kouni_UAV-02_1_\ndoi:1...,"title:""Orthophoto & DEM (MNE) issues d'images ...","abstract:""Ce jeu de données présente les résul...","theme[General]:Comores,Grande Comore,flooding,...","author:sylvain.poulain@ird.fr,julien.barde@ird...",publication:2023-10-11_\nedition:2023-10-11,dataset,fra,NaN,2023-06-27,thumbnail:G2OI@https://g2oi.ird.fr/static/img/...,useLimitation:Utilisation libre sous réserve d...,"statement:""- Camera model and parameters:\n Ma...",resource:image/tiff_\ndistribution:image/tiff[...,source:000_photogrammetry_report.pdf@/home/syl...
1,id:20230701_COM-nioumadzaha-mitsoudje_UAV-02_1...,"title:""Orthophoto & DEM (MNE) issues d'images ...","abstract:""Ce jeu de données présente les résul...","theme[General]:Comores,Grande Comore,flooding,...","author:sylvain.poulain@ird.fr,julien.barde@ird...",publication:2023-10-11_\nedition:2023-10-11,dataset,fra,NaN,2023-06-26 15-59-22 - 2023-07-01 11-36-44,thumbnail:G2OI@https://g2oi.ird.fr/static/img/...,useLimitation:Utilisation libre sous réserve d...,"statement:""- Camera model and parameters:\n Ma...",resource:image/tiff_\ndistribution:image/tiff[...,source:000_photogrammetry_report.pdf@/home/syl...
2,id:20230626_COM-bandamaji_UAV-01_2_\ndoi:10.52...,"title:""Orthophoto & DEM (MNE) issues d'images ...","abstract:""Ce jeu de données présente les résul...","theme[General]:Comores,Grande Comore,flooding,...","author:sylvain.poulain@ird.fr,julien.barde@ird...",publication:2023-10-11_\nedition:2023-10-11,dataset,fra,NaN,2023-06-26 15-51-39 - 2023-06-26 15-58-55,thumbnail:G2OI@https://g2oi.ird.fr/static/img/...,useLimitation:Utilisation libre sous réserve d...,"statement:""- Camera model and parameters:\n Ma...",resource:image/tiff_\ndistribution:image/tiff[...,source:000_photogrammetry_report.pdf@/home/syl...
3,id:20230701_COM-nioumadzaha-mitsoudje_UAV-01_3...,"title:""Orthophoto & DEM (MNE) issues d'images ...","abstract:""Ce jeu de données présente les résul...","theme[General]:Comores,Grande Comore,flooding,...","author:sylvain.poulain@ird.fr,julien.barde@ird...",publication:2023-10-11_\nedition:2023-10-11,dataset,fra,NaN,2023-07-01 11-26-01 - 2023-07-01 11-34-29,thumbnail:G2OI@https://g2oi.ird.fr/static/img/...,useLimitation:Utilisation libre sous réserve d...,"statement:""- Camera model and parameters:\n Ma...",resource:image/tiff_\ndistribution:image/tiff[...,source:000_photogrammetry_report.pdf@/home/syl...
4,id:20230626_COM-mitsamiouli_UAV-01_1_\ndoi:10....,"title:""Orthophoto & DEM (MNE) issues d'images ...","abstract:""Ce jeu de données présente les résul...","theme[General]:Comores,Grande Comore,flooding,...","author:sylvain.poulain@ird.fr,julien.barde@ird...",publication:2023-10-11_\nedition:2023-10-11,dataset,fra,NaN,2023-06-27 11-58-56 - 2023-06-27 12-15-51,thumbnail:G2OI@https://g2oi.ird.fr/static/img/...,useLimitation:Utilisation libre sous réserve d...,"statement:""- Camera model and parameters:\n Ma...",resource:image/tiff_\ndistribution:image/tiff[...,source:000_photogrammetry_report.pdf@/home/syl...


### Update Metadata

In [52]:
### Update Metadata
print("#### Update metadata")
base_url = "https://zenodo.org/api/"
ACCESS_TOKEN='lY0lha8RfhJrXe5Wb3Y1zRcqEZAZkoe5zQC8shhWP30CQPzX6voV9Ak2cJOG'

for zipul in range(len(df)):
    print('\nDataset:', zipul+1, "/" ,len(df), "\n    ",df.iloc[zipul]['Identifier'])
    zenodo_baseurl = base_url 
    
    if 'doi:' in df.iloc[zipul]['Identifier']:
        doi_raw = df.iloc[zipul]['Identifier'].split('_\ndoi:')[1].split('_\n')[0].split('.')[-1]
        print("#########\n#" + doi_raw + "#\n#########")
        # print("\nDOI already present: ",doi_raw)
        # getrecid = zenlist_single(zenodo_baseurl, ACCESS_TOKEN, str(doi_raw))
        # print(getrecid.text)
        
        # zenval = zenvar(getrecid)
        # print(zenval)
        # print("\nEnrich upload with metadata")
        zenedit(zenodo_baseurl, ACCESS_TOKEN, doi_raw)
        zen_metadata = zenmdt(zenodo_baseurl, ACCESS_TOKEN, doi_raw, df, zipul, contact_file="/home/sylvain/DroneMDT/geoflow_g2oi_contacts.csv")
        print(zen_metadata)
        print(zen_metadata.reason)

    ### Publish to Zenodo
    print("Publish record")
    published = zenpublish(zenodo_baseurl, ACCESS_TOKEN, doi_raw)
    print(published.text)

#### Update metadata

Dataset: 1 / 17 
     id:20230626_COM-bangoua-kouni_UAV-02_1_
doi:10.5281/zenodo.8431136_
conceptdoi:10.5281/zenodo.8431135
#########
#8431136#
#########
Using contact file
  Looking for sylvain.poulain@ird.fr
    Look for email sylvain.poulain@ird.fr in Contact file
      Add orcid
      Add First and Last Name
      Add Affiliation
  Looking for julien.barde@ird.fr
    Look for email julien.barde@ird.fr in Contact file
      Add orcid
      Add First and Last Name
      Add Affiliation
  Looking for natali.mmadi@gmail.com
    Look for email natali.mmadi@gmail.com in Contact file
      Add First and Last Name
      Add Affiliation
  Looking for mohamedhamidou957@gmail.com
    Look for email mohamedhamidou957@gmail.com in Contact file
      Add First and Last Name
      Add Affiliation
  Looking for comores.dgsc@gmail.com
    Look for email comores.dgsc@gmail.com in Contact file
      Add First and Last Name
      Add Affiliation
{'metadata': {'title': 'Images dro

In [120]:
### Update Processed Metadata
print("#### Update metadata")
base_url = "https://zenodo.org/api/"
ACCESS_TOKEN='lY0lha8RfhJrXe5Wb3Y1zRcqEZAZkoe5zQC8shhWP30CQPzX6voV9Ak2cJOG'

list_all = zenlist_all(zenodo_baseurl, ACCESS_TOKEN).json()

for zipul in range(len(df)):
    print('\nDataset:', zipul+1, "/" ,len(df), "\n    ",df.iloc[zipul]['Identifier'])
    zenodo_baseurl = base_url 
    
    if 'doi:' in df.iloc[zipul]['Identifier']:
        doi_raw = df.iloc[zipul]['Identifier'].split('_\ndoi:')[1].split('_\n')[0].split('.')[-1]
        print("#########\n#" + doi_raw + "#\n#########")
        # print("\nDOI already present: ",doi_raw)
        # getrecid = zenlist_single(zenodo_baseurl, ACCESS_TOKEN, str(doi_raw))
        # print(getrecid.text)
        
        # zenval = zenvar(getrecid)
        # print(zenval)
        # print("\nEnrich upload with metadata")
        zenedit(zenodo_baseurl, ACCESS_TOKEN, doi_raw)
        zen_metadata = zenmdt(zenodo_baseurl, ACCESS_TOKEN, doi_raw, df, zipul, contact_file="/home/sylvain/DroneMDT/geoflow_g2oi_contacts.csv")
        print(zen_metadata)
        print(zen_metadata.reason)

    ### Publish to Zenodo
    print("Publish record")
    published = zenpublish(zenodo_baseurl, ACCESS_TOKEN, doi_raw)
    print(published.text)

#### Update metadata

Dataset: 1 / 17 
     id:20230626_COM-bangoua-kouni_UAV-02_1_
doi:10.5281/zenodo.8436903
#########
#8436903#
#########
Using contact file
  Looking for sylvain.poulain@ird.fr
    Look for email sylvain.poulain@ird.fr in Contact file
      Add orcid
      Add First and Last Name
      Add Affiliation
  Looking for julien.barde@ird.fr
    Look for email julien.barde@ird.fr in Contact file
      Add orcid
      Add First and Last Name
      Add Affiliation
  Looking for natali.mmadi@gmail.com
    Look for email natali.mmadi@gmail.com in Contact file
      Add First and Last Name
      Add Affiliation
  Looking for mohamedhamidou957@gmail.com
    Look for email mohamedhamidou957@gmail.com in Contact file
      Add First and Last Name
      Add Affiliation
  Looking for comores.dgsc@gmail.com
    Look for email comores.dgsc@gmail.com in Contact file
      Add First and Last Name
      Add Affiliation
{'metadata': {'title': '"Orthophoto & DEM (MNE) issues d\'images de d

In [83]:
zen_latest(zenodo_baseurl, ACCESS_TOKEN, doi_raw).url

'https://zenodo.org/api/records/8436453/versions/latest?access_token=lY0lha8RfhJrXe5Wb3Y1zRcqEZAZkoe5zQC8shhWP30CQPzX6voV9Ak2cJOG'

In [117]:
list_all = zenlist_all(zenodo_baseurl, ACCESS_TOKEN).json()
print(doi_raw)
for doi in range(len(list_all)):
    doi_raw in list_all[doi]
    try:
        print(list_all[doi]['metadata']['related_identifiers'][0]["identifier"].replace("urn:", "id:") + "_\ndoi:" + str(list_all[doi]["id"]))
        # print(list_all[doi]["id"])
    except:
        print("nok")

8436453
id:20230701_COM-nioumadzaha-mitsoudje_UAV-02_1_
doi:8436907
nok
10.5281/zenodo.10029217_
doi:10029199
10.5281/zenodo.10029217_
doi:10023277
10.5281/zenodo.10029217_
doi:10019809
id:20230701_COM-nioumadzaha-mitsoudje_UAV-01_1_
doi:8436939
10.5281/zenodo.10029217_
doi:10014805
10.5281/zenodo.10014573_
doi:10014783
10.5281/zenodo.10014573_
doi:10014748
10.5281/zenodo.10014573_
doi:10014728
nok
10.5281/zenodo.10029217_
doi:10002758
id:20230627_COM-vouvouni_UAV-02_2_
doi:8436942
id:20230626_COM-mitsamiouli_UAV-01_2_
doi:8436965
id:20230628_COM-maloudja_UAV-02_2_
doi:8436964
id:20230626_COM-bandamaji_UAV-01_1_
doi:8436963
id:20230626_COM-bandamaji_UAV-02_1_
doi:8436959
id:20230626_COM-bangoua-kouni_UAV-01_1_
doi:8436956
id:20230626_COM-mitsamiouli_UAV-02_1_
doi:8436954
id:20230627_COM-vouvouni_UAV-01_2_
doi:8436955
id:20230627_COM-vouvouni_UAV-02_1_
doi:8436952
id:20230627_COM-vouvouni_UAV-01_1_
doi:8436947
id:20230701_COM-nioumadzaha-mitsoudje_UAV-01_2_
doi:8436945
id:20230701_COM-n

In [69]:
zenlist_single(zenodo_baseurl, ACCESS_TOKEN, doi_raw).json()["links"]

{'self': 'https://zenodo.org/api/records/8436453',
 'html': 'https://zenodo.org/records/8436453',
 'doi': 'https://doi.org/10.5281/zenodo.8436453',
 'parent_doi': 'https://zenodo.org/doi/10.5281/zenodo.8436452',
 'badge': 'https://zenodo.org/badge/doi/10.5281%2Fzenodo.8436453.svg',
 'conceptbadge': 'https://zenodo.org/badge/doi/10.5281%2Fzenodo.8436452.svg',
 'files': 'https://zenodo.org/api/records/8436453/files',
 'bucket': 'https://zenodo.org/api/files/509cacb0-5674-4dbe-9787-eee9cf21e057',
 'thumb250': 'https://zenodo.org/record/8436453/thumb250',
 'thumbs': {'10': 'https://zenodo.org/record/8436453/thumb10',
  '50': 'https://zenodo.org/record/8436453/thumb50',
  '100': 'https://zenodo.org/record/8436453/thumb100',
  '250': 'https://zenodo.org/record/8436453/thumb250',
  '750': 'https://zenodo.org/record/8436453/thumb750',
  '1200': 'https://zenodo.org/record/8436453/thumb1200'},
 'latest_draft': 'https://zenodo.org/api/deposit/depositions/8436453',
 'latest_draft_html': 'https://z

In [47]:
### Check RAW_DATA DOI's
for doi in range(len(df)):
    print(df.iloc[doi]["Identifier"].replace("_\n", ","))

id:20230626_COM-bangoua-kouni_UAV-02_1,doi:10.5281/zenodo.8431136,conceptdoi:10.5281/zenodo.8431135
id:20230701_COM-nioumadzaha-mitsoudje_UAV-02_1,doi:10.5281/zenodo.8431203,conceptdoi:10.5281/zenodo.8431202
id:20230626_COM-bandamaji_UAV-01_2,doi:10.5281/zenodo.8431498,conceptdoi:10.5281/zenodo.8431497
id:20230701_COM-nioumadzaha-mitsoudje_UAV-01_3,doi:10.5281/zenodo.8431531,conceptdoi:10.5281/zenodo.8431530
id:20230626_COM-mitsamiouli_UAV-01_1,doi:10.5281/zenodo.8431634,conceptdoi:10.5281/zenodo.8431633
id:20230701_COM-nioumadzaha-mitsoudje_UAV-01_1,doi:10.5281/zenodo.8431736,conceptdoi:10.5281/zenodo.8431735
id:20230627_COM-vouvouni_UAV-02_2,doi:10.5281/zenodo.8431807,conceptdoi:10.5281/zenodo.8431806
id:20230701_COM-nioumadzaha-mitsoudje_UAV-01_2,doi:10.5281/zenodo.8431876,conceptdoi:10.5281/zenodo.8431875
id:20230627_COM-vouvouni_UAV-01_1,doi:10.5281/zenodo.8431985,conceptdoi:10.5281/zenodo.8431984
id:20230627_COM-vouvouni_UAV-02_1,doi:10.5281/zenodo.8432062,conceptdoi:10.5281/zeno

In [119]:
### Check PROCESSED_DATA DOI's
for doi in range(len(df)):
    print(df.iloc[doi]["Identifier"].replace("_\n", ","))

id:20230626_COM-bangoua-kouni_UAV-02_1,doi:10.5281/zenodo.8436903
id:20230701_COM-nioumadzaha-mitsoudje_UAV-02_1,doi:10.5281/zenodo.8436907
id:20230626_COM-bandamaji_UAV-01_2,doi:10.5281/zenodo.8436931
id:20230701_COM-nioumadzaha-mitsoudje_UAV-01_3,doi:10.5281/zenodo.8436934
id:20230626_COM-mitsamiouli_UAV-01_1,doi:10.5281/zenodo.8436935
id:20230701_COM-nioumadzaha-mitsoudje_UAV-01_1,doi:10.5281/zenodo.8436939
id:20230627_COM-vouvouni_UAV-02_2,doi:10.5281/zenodo.8436942
id:20230701_COM-nioumadzaha-mitsoudje_UAV-01_2,doi:10.5281/zenodo.8436945
id:20230627_COM-vouvouni_UAV-01_1,doi:10.5281/zenodo.8436947
id:20230627_COM-vouvouni_UAV-02_1,doi:10.5281/zenodo.8436952
id:20230626_COM-mitsamiouli_UAV-02_1,doi:10.5281/zenodo.8436954
id:20230627_COM-vouvouni_UAV-01_2,doi:10.5281/zenodo.8436955
id:20230626_COM-bangoua-kouni_UAV-01_1,doi:10.5281/zenodo.8436956
id:20230626_COM-bandamaji_UAV-02_1,doi:10.5281/zenodo.8436959
id:20230626_COM-bandamaji_UAV-01_1,doi:10.5281/zenodo.8436963
id:20230628_CO